In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones de Pedro Sanchez.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 20  # Longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir el modelo LSTM con 2 capas y optimizaciones
vocab_size = len(vocab)
embedding_dim = 200  # Dimensión del embedding
lstm_units = 512  # Número de unidades en LSTM

def build_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=sequence_length),
        LSTM(lstm_units, return_sequences=True),
        Dropout(0.3),
        LSTM(lstm_units),
        Dropout(0.3),
        Dense(vocab_size, activation='softmax')
    ])
    return model

model = build_model()

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Guardar el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint('modelo_LSTM_2capas_Sanchez_optimizado.keras', monitor='loss', save_best_only=True)

# Entrenar el modelo con un número moderado de épocas
print("Entrenando el modelo...")
model.fit(X, y, epochs=50, batch_size=128, verbose=1, callbacks=[checkpoint])

# Función para generar texto con control de temperatura
def generate_text(model, start_sequence, max_words=150, temperature=1.0):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

    return generated

# Interactuar con el modelo generando texto
print("¡Modelo entrenado! Puedes interactuar con él para generar texto.")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Generando texto...")
    print(generate_text(model, prompt, max_words=150, temperature=0.8))

Por favor, sube el archivo .txt con las intervenciones de Pedro Sanchez.


Saving intervencionesSanchez.txt to intervencionesSanchez.txt


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Entrenando el modelo...
Epoch 1/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.0497 - loss: 7.1652
Epoch 2/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0533 - loss: 6.1986
Epoch 3/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0586 - loss: 6.1882
Epoch 4/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0622 - loss: 6.1199
Epoch 5/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0802 - loss: 5.9931
Epoch 6/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0868 - loss: 5.9175
Epoch 7/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0920 - loss: 5.8241
Epoch 8/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0983 - loss: 5.7175
Epoch 9/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0978 - loss: 5.6438
Epoch 10/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.1044 - loss: 5.5784
Epoch 11/50
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.1043 - loss: 5.5068
Epoch 12/50
308/3